In [ ]:
### Automl for daily forecasting

In [ ]:
# import packages
import numpy as np
import pandas as pd
import statsmodels as sm
from matplotlib import pyplot as plt
## mlflow integration for model experimentation
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import shap
import seaborn as sns
# from pylab import rcParams
# from sklearn.linear_model import LinearRegression
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa import api as smt
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.stattools import adfuller
import matplotlib as mpl
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")
#modeling packages
#from tqdm import tqdm
#from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
import metric
from metric import evaluate
#complex amra model
# from statsmodels.tsa.ar_model import AR
# from statsmodels.tsa.arima_model import ARIMA, ARMA
# from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
# pm model 
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six
# import mlrose
# import joblib
# sys.modules['sklearn.externals.joblib'] = joblib
# import pmdarima as pm
# from statsmodels.tsa.statespace.sarimax import SARIMAX
#ensemble modeling
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


import plots 
from plots import plot_autocorrelation_grid, plot_grid
import eng_feature 
from eng_feature import check_df,lag_features,roll_mean_features,ewm_features,create_time_features

In [ ]:
# stat forecaster use
import random
import time
from multiprocessing import cpu_count, Pool # for prophet
from itertools import product
import matplotlib.pyplot as plt
from statsforecast.core import StatsForecast
from statsforecast.models import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import ParameterGrid

from statsforecast.models import (
    adida, 
    croston_classic, 
    croston_sba, 
    croston_optimized,
    historic_average,
    imapa,
    naive,
    random_walk_with_drift, 
    seasonal_exponential_smoothing,
    seasonal_naive, 
    seasonal_window_average,
    ses, 
    tsb,
    window_average
)


In [ ]:
date_column ='EffectiveDate'
file_location = 'FleetForecasting_Top100ProductSubCategory_WithIHSData_Weather_BYDay_V2.csv'
input_data_raw=pd.read_csv(file_location ,sep ='|',parse_dates=[date_column])
input_data_copy = input_data_raw.copy()

In [ ]:
input_data_copy.head()

In [ ]:
#input_data_copy=input_data_copy[input_data_copy["Division"].isin([502,503,504,510,511])]

In [ ]:
input_data_copy.columns

In [ ]:
input_data_copy.shape

In [ ]:
input_data_copy.head()

In [ ]:
input_data_copy[['EffectiveDate','ProductCategory_Nbl','ProductCategory_Desc','Division','DRKey']].nunique() 
# list out amount of unqiue data for each column - nbl and desc is missing one eq?

In [ ]:
input_data_copy.sort_values(by=['EffectiveDate'], ascending=True,inplace=True)

In [ ]:
input_data_copy.head()

In [ ]:
input_data_copy[date_column].min(), input_data_copy[date_column].max()  

In [ ]:
input_data_copy["OnRent"].describe([0.10, 0.30, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99])

In [ ]:
input_data_copy['clean_time']=pd.to_datetime(input_data_copy[date_column]) # do i need this? 
input_data_copy.sort_values(by=['DRKey',date_column], ascending=False,inplace=True)
input_data_copy['ProductCategory_Desc'] = input_data_copy['ProductCategory_Desc'].str.replace(r"[\"\',< ]", '')
cols = ['ProductCategory_Nbl', 'ProductCategory_Desc']
input_data_copy['eq_nm'] = input_data_copy[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
input_data_copy = lag_features(input_data_copy, [91, 98, 105, 112, 119, 126, 182, 364])
check_df(input_data_copy)
# input_data_copy['OnRent_diff']=input_data_copy.groupby('DRKey')['OnRent'].diff(7)
# input_data_copy['AvgTemp_diff'] = input_data_copy.groupby('DRKey')['AvgTemp'].diff(7)
# input_data_copy['AvgPrcp_diff'] = input_data_copy.groupby('DRKey')['AvgPrcp'].diff(7) 
# establish difference with 1 month period rolling for stationarity

In [ ]:
input_data_copy = roll_mean_features(input_data_copy, [365, 546])
input_data_copy.tail()

In [ ]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365]
input_data_copy = ewm_features(input_data_copy, alphas, lags)
check_df(input_data_copy)

In [ ]:
#input_data_copy['OnRent'] = np.log1p(input_data_copy["OnRent"].values)

In [ ]:
input_data_copy.groupby(["Division", "eq_nm"]).agg({"OnRent": ["sum", "mean", "median", "std"]})

In [ ]:
feature_importance = [col for col in input_data_copy.columns if col not in ['RegionName','clean_time','Rental', 'QtyOwned', "ProductCategory_Desc",'ProductCategory_Nbl']]

In [ ]:
feature_importance

In [ ]:
input_data_copy.head()

In [ ]:
#feature_importance=['OnRent',date_column,'eq_nm','Division','IHSRentalMarket','AvgTemp','AvgPrcp'] # not adding differencing just yet
sg_district_elect=input_data_copy.copy()
sg_district_elect_train=sg_district_elect[feature_importance]

In [ ]:
sg_district_elect_train.head()

In [ ]:
sg_district_elect_train.drop(sg_district_elect_train.loc[sg_district_elect_train[date_column] < '2015-05-01 00:00:00'].index, inplace=True) # prob dont need because data is from 2016
# drop 2020 for covid
sg_district_elect_train.drop(sg_district_elect_train.loc[sg_district_elect_train[date_column] > '2020-02-01 01:00:00'].index, inplace=True)

In [ ]:
sg_district_elect_train.set_index(date_column,inplace=True)
sg_district_elect_train.sort_index(inplace=True)
#sg_district_elect_train.dropna(inplace=True)

In [ ]:
sg_district_elect_train.head()

In [ ]:
sg_training = sg_district_elect_train.copy()
split_date = '2019-5-20'
df_training = sg_training.loc[sg_training.index <= split_date]
df_test = sg_training.loc[sg_training.index > split_date]

## Add time features for date_column

In [ ]:
categorical_features =['eq_nm','Division','Region']
target = 'OnRent'

In [ ]:
sg_district_elect_train=pd.get_dummies(sg_district_elect_train,columns=categorical_features)

In [ ]:
sg_training=sg_district_elect_train
# differencing and dropping first index
#sg_training = sg_training.diff(periods=1)
#sg_training.drop(index=sg_training.index[0], axis=0, inplace=True)
split_date = '2019-5-20'
df_training = sg_training.loc[sg_training.index <= split_date]
df_test = sg_training.loc[sg_training.index > split_date]
print(f"{len(df_training)} months of training data \n {len(df_test)} months of testing data ")

In [ ]:
df_training.index.min(), df_training.index.max()

In [ ]:
# include oberservations - list out all the stuff out

In [ ]:
df_test.index.min(), df_test.index.max() #8 months of testing out -

In [ ]:
X_train_df, y_train = create_time_features(
    df_training, target=target)

X_test_df, y_test = create_time_features(df_test, target=target)

numerical_data = ['hour',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'sin_day',
 'cos_day',
 'dayofmonth',
 'weekofyear']

categorical_data=set(X_train_df.columns)-set(numerical_data)
print(f'I have {len(categorical_data)} of categorical data and {len(numerical_data)} numerical features') # inaccurate have to add ihs and other diff - no point for this fstring
##TODO remove fstring find out easier way to identify numerical data - data values more than 2???

In [ ]:
# ct = ColumnTransformer([
#         ('standard', StandardScaler(),numerical_data) #fit numerical data
#     ], remainder='passthrough')

# X_train=ct.fit_transform(X_train_df)
# X_test=ct.fit_transform(X_test_df) # am I losing information by fitting time? may not use this ..

resultsDict = {}
predictionsDict = {}

In [ ]:
X_train_df.head(2)

In [ ]:
X_test_df.head(2)

## Lightgbm Model and Diagnostic Charts

In [ ]:
## TODO Cross Validation/ Grid Search parameters for more defined trees
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
EXPERIMENT_NAME = 'On_Rent_Time_Series'
mlflow.set_runName = 'lgbm predictions'# doesnt do anything
client = MlflowClient()
experiments = client.list_experiments()
if EXPERIMENT_NAME not in [e.name for e in experiments]:
    mlflow.create_experiment(EXPERIMENT_NAME)

In [ ]:
mlflow.set_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name='simple_light'):
    lightGBM = lgbm.LGBMRegressor()
    lightGBM.fit(X_train_df.drop(columns='DRKey'), y_train)
    yhat = lightGBM.predict(X_test_df.drop(columns='DRKey'))
    resultsDict['Lightgbm'] = evaluate(df_test.OnRent, yhat)
    predictionsDict['Lightgbm'] = yhat
    mlflow.log_metrics(evaluate(df_test.OnRent, yhat))
    signature = infer_signature(X_train_df.drop(columns='DRKey'), lightGBM.predict(X_test_df.drop(columns='DRKey')))
    mlflow.lightgbm.log_model(lightGBM, "lgbm", signature = signature)
    ## general diagnostics - 
    plt.plot(df_test.OnRent.values, label='Original')
    plt.plot(yhat, color='red', label='lgbm')
    plt.legend()
    plt.savefig('images/AVP.png')
    plt.clf()
    shap_values = shap.TreeExplainer(lightGBM).shap_values(X_train_df.drop(columns='DRKey'))
    shap.summary_plot(shap_values, X_train_df.drop(columns='DRKey'),show=False)
    plt.savefig('images/shap.png',bbox_inches='tight')
    plt.clf()
    feature_imp = pd.DataFrame(sorted(zip(lightGBM.feature_importances_,X_train_df.drop(columns='DRKey').columns)), columns=['Value','Feature'])
    plt.figure(figsize=(20, 10))
    data=feature_imp.sort_values(by="Value", ascending=False).head(30)
    sns.barplot(x="Value", y="Feature", data=data)
    plt.title('LightGBM Features (avg over folds)')
    plt.savefig('images/lgbm_importances-01.png')
    plt.clf()
    mlflow.log_artifacts("images", artifact_path="diagnostics")
mlflow.end_run

## Forecaster for univerariate models 

#### Dont run if you dont have appropriate conda env - will break azure auto ml

In [ ]:
#df_test=df_test[['EffectiveDate','DRKey','lgbm','OnRent']]

In [ ]:
forecaster_data=input_data_copy.copy()

In [ ]:
#forecaster_data.drop(forecaster_data.loc[forecaster_data[date_column] < '2015-05-01 00:00:00'].index, inplace=True) # prob dont need because data is from 2016
forecaster_data.drop(forecaster_data.loc[forecaster_data[date_column] > '2020-02-01 01:00:00'].index, inplace=True)
forecaster_data=forecaster_data[['DRKey','EffectiveDate','OnRent']]

In [ ]:
forecaster_data.rename(columns={'DRKey': 'unique_id'},inplace = True)
forecaster_data.rename(columns={'EffectiveDate': 'ds'},inplace = True)
forecaster_data.rename(columns={'OnRent': 'y'},inplace = True)
date_column = 'ds' #changing column name for stat forecast
df_forecaster_training = forecaster_data.loc[forecaster_data[date_column] <= split_date]
df_forecaster_test = forecaster_data.loc[forecaster_data[date_column] > split_date]

In [ ]:
df_forecaster_training.head()

In [ ]:
n_series = 1
seasonality = 70
models = [
    naive,
    historic_average,
    random_walk_with_drift,
    (seasonal_exponential_smoothing, seasonality, 0.6),
    (seasonal_naive, seasonality),
    auto_arima
]
fcst = StatsForecast(df_forecaster_training.set_index('unique_id'), models=models, freq='D', 
                     n_jobs=min(cpu_count(), n_series))

In [ ]:
init = time.time()
with np.errstate(invalid='ignore'):
    forecasts = fcst.forecast(256)
end = time.time()

time_nixtla = end - init
time_nixtla

In [ ]:
forecasts = forecasts.reset_index()

In [ ]:
df_test_merge=df_test.merge(forecasts, how='left', left_on=['DRKey','EffectiveDate'],right_on=['unique_id', 'ds'])

In [ ]:
df_test_merge.drop(columns =['unique_id','ds'], inplace = True)

In [ ]:
df_test_merge.head()

In [ ]:
error_metric={}
for x in df_test_merge.columns :
    if(x not in ['EffectiveDate','DRKey','OnRent']):
        error_metric[x]=(evaluate(df_test_merge.OnRent, df_test_merge[x]))

In [ ]:
error_metric

In [ ]:
#df_test_merge.to_csv('forecast_lightgbm_arima.csv')